In [1]:
import tensorflow as tf
import pathlib
import os
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
np.set_printoptions(precision=4)

2024-04-24 07:31:29.327307: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Basic mechanics

* [`tf.data.Dataset.from_tensors()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensors)
* [`tf.data.Dataset.from_tensor_slices()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices)
* [`tf.data.TFRecordDataset()`](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset): Alternatively, if your input data is stored in a file in the recommended TFRecord format, you can use 


In [2]:
dataset = tf.data.Dataset.from_tensor_slices([8, 3, 0, 8, 2, 1])
dataset

2024-04-24 07:31:30.552575: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-04-24 07:31:30.552595: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:134] retrieving CUDA diagnostic information for host: 0b7292f20506
2024-04-24 07:31:30.552600: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:141] hostname: 0b7292f20506
2024-04-24 07:31:30.552654: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:165] libcuda reported version is: 545.29.6
2024-04-24 07:31:30.552667: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:169] kernel reported version is: 545.29.6
2024-04-24 07:31:30.552670: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:248] kernel version seems to match DSO: 545.29.6


<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int32, name=None)>

In [3]:
for elem in dataset:
  print(elem.numpy())

8
3
0
8
2
1


2024-04-24 07:31:30.581124: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [4]:
it = iter(dataset)

print(next(it).numpy())

8


In [5]:
print(dataset.reduce(0, lambda state, value: state + value).numpy())

22


### Dataset structure

A dataset produces a sequence of *elements*, where each element is the same (nested) structure of *components*. Individual components of the structure can be of any type representable by [`tf.TypeSpec`](https://www.tensorflow.org/api_docs/python/tf/TypeSpec), including [`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor), [`tf.sparse.SparseTensor`](https://www.tensorflow.org/api_docs/python/tf/sparse/SparseTensor), [`tf.RaggedTensor`](https://www.tensorflow.org/api_docs/python/tf/RaggedTensor), [`tf.TensorArray`](https://www.tensorflow.org/api_docs/python/tf/TensorArray), or [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

The [`Dataset.element_spec`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#element_spec) property allows you to inspect the type of each element component. The property returns a *nested structure* of [`tf.TypeSpec`](https://www.tensorflow.org/api_docs/python/tf/TypeSpec) objects, matching the structure of the element, which may be a single component, a tuple of components, or a nested tuple of components. For example:

In [6]:
dataset1 = tf.data.Dataset.from_tensor_slices(tf.random.uniform([4, 10]))

dataset1.element_spec

TensorSpec(shape=(10,), dtype=tf.float32, name=None)

In [7]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))

dataset2.element_spec

(TensorSpec(shape=(), dtype=tf.float32, name=None),
 TensorSpec(shape=(100,), dtype=tf.int32, name=None))

In [8]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3.element_spec

(TensorSpec(shape=(10,), dtype=tf.float32, name=None),
 (TensorSpec(shape=(), dtype=tf.float32, name=None),
  TensorSpec(shape=(100,), dtype=tf.int32, name=None)))

In [9]:
# Dataset containing a sparse tensor.
dataset4 = tf.data.Dataset.from_tensors(tf.SparseTensor(indices=[[0, 0], [1, 2]], values=[1, 2], dense_shape=[3, 4]))

dataset4.element_spec

SparseTensorSpec(TensorShape([3, 4]), tf.int32)

In [10]:
# Use value_type to see the type of value represented by the element spec
dataset4.element_spec.value_type

tensorflow.python.framework.sparse_tensor.SparseTensor

The `Dataset` transformations support datasets of any structure. When using the [`Dataset.map`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#map), and [`Dataset.filter`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#filter) transformations, which apply a function to each element, the element structure determines the arguments of the function:

In [11]:
dataset1 = tf.data.Dataset.from_tensor_slices(
    tf.random.uniform([4, 10], minval=1, maxval=10, dtype=tf.int32))

dataset1

<_TensorSliceDataset element_spec=TensorSpec(shape=(10,), dtype=tf.int32, name=None)>

In [12]:
for z in dataset1:
  print(z.numpy())

[4 3 5 4 6 7 9 9 4 4]
[5 5 8 3 1 2 3 6 4 4]
[3 6 8 1 5 2 8 6 2 8]
[2 2 4 7 9 1 2 4 9 9]


2024-04-24 07:31:30.630764: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [13]:
dataset2 = tf.data.Dataset.from_tensor_slices(
   (tf.random.uniform([4]),
    tf.random.uniform([4, 100], maxval=100, dtype=tf.int32)))

dataset2

<_TensorSliceDataset element_spec=(TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None))>

In [14]:
dataset3 = tf.data.Dataset.zip((dataset1, dataset2))

dataset3

<_ZipDataset element_spec=(TensorSpec(shape=(10,), dtype=tf.int32, name=None), (TensorSpec(shape=(), dtype=tf.float32, name=None), TensorSpec(shape=(100,), dtype=tf.int32, name=None)))>

In [15]:
for a, (b,c) in dataset3:
  print('shapes: {a.shape}, {b.shape}, {c.shape}'.format(a=a, b=b, c=c))

shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)
shapes: (10,), (), (100,)


2024-04-24 07:31:30.644184: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


## Reading input data

### Consuming NumPy arrays (`tf.data.Dataset.from_tensor_slices`)

Refer to the [Loading NumPy arrays](https://www.tensorflow.org/tutorials/load_data/numpy) tutorial for more examples.

If all of your input data fits in memory, the simplest way to create a `Dataset` from them is to convert them to [`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) objects and use [`Dataset.from_tensor_slices`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#from_tensor_slices).

In [16]:
train, test = tf.keras.datasets.fashion_mnist.load_data()

In [17]:
images, labels = train
images = images/255

dataset = tf.data.Dataset.from_tensor_slices((images, labels))
dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(28, 28), dtype=tf.float64, name=None), TensorSpec(shape=(), dtype=tf.uint8, name=None))>

In [18]:
images.shape, labels.shape

((60000, 28, 28), (60000,))

 This works well for a small dataset, but wastes memory---because the contents of the array will be copied multiple times---and can run into the 2GB limit for the `tf.GraphDef` protocol buffer.

### Consuming Python generators (`tf.data.Dataset.from_generator`)

Another common data source that can easily be ingested as a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset) is the python generator.

In [19]:
def count(stop):
  i = 0
  while i<stop:
    yield i
    i += 1

In [20]:
for n in count(5):
  print(n)

0
1
2
3
4


In [21]:
ds_counter = tf.data.Dataset.from_generator(count, args=[25], output_types=tf.int32, output_shapes = (), )

In [22]:
for count_batch in ds_counter.repeat().batch(10).take(10):
  print(count_batch.numpy())

[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]
[0 1 2 3 4 5 6 7 8 9]
[10 11 12 13 14 15 16 17 18 19]
[20 21 22 23 24  0  1  2  3  4]
[ 5  6  7  8  9 10 11 12 13 14]
[15 16 17 18 19 20 21 22 23 24]


2024-04-24 07:31:31.145008: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


The `output_shapes` argument is not *required* but is highly recommended as many TensorFlow operations do not support tensors with an unknown rank. If the length of a particular axis is unknown or variable, set it as `None` in the `output_shapes`.

It's also important to note that the `output_shapes` and `output_types` follow the same nesting rules as other dataset methods.

Here is an example generator that demonstrates both aspects: it returns tuples of arrays, where the second array is a vector with unknown length.

In [23]:
def gen_series():
  i = 0
  while True:
    size = np.random.randint(0, 10)
    yield i, np.random.normal(size=(size,))
    i += 1

In [24]:
for i, series in gen_series():
  print(i, ":", str(series))
  if i > 5:
    break

0 : [-1.3876  0.8396  1.879  -0.8719  0.7578  0.7361]
1 : [0.1993 0.3532]
2 : [-0.2315 -1.9471  0.8528 -1.0563 -1.3822 -0.848   0.0339  1.0113  1.6708]
3 : []
4 : [-1.2596 -0.4125 -1.1296]
5 : [-0.2018  1.3765  0.4575 -0.6453 -0.0341 -1.7881  0.2926]
6 : [-0.674  -1.7723 -1.2631  2.3173 -0.8635 -0.435  -0.9496]


The first output is an `int32` the second is a `float32`.

The first item is a scalar, shape `()`, and the second is a vector of unknown length, shape `(None,)`

In [25]:
ds_series = tf.data.Dataset.from_generator(
    gen_series,
    output_types=(tf.int32, tf.float32),
    output_shapes=((), (None,)))

ds_series

<_FlatMapDataset element_spec=(TensorSpec(shape=(), dtype=tf.int32, name=None), TensorSpec(shape=(None,), dtype=tf.float32, name=None))>

Now it can be used like a regular [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset). Note that when batching a dataset with a variable shape, you need to use [`Dataset.padded_batch`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#padded_batch).

In [26]:
ds_series_batch = ds_series.shuffle(20).padded_batch(10)

ids, sequence_batch = next(iter(ds_series_batch))
print(ids.numpy())
print()
print(sequence_batch.numpy())

[ 9 14 15 21  1 19  7 16  0  8]

[[ 0.7579 -0.4827  0.3717  0.      0.      0.      0.      0.      0.    ]
 [-0.5983  0.      0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.9442 -0.6436 -2.4037  1.162   0.      0.      0.      0.      0.    ]
 [-0.4544  0.7663 -0.9616 -0.654  -0.2451 -0.5311  0.      0.      0.    ]
 [-2.2924 -0.9098  0.      0.      0.      0.      0.      0.      0.    ]
 [ 0.1162  1.0189  0.0063  1.7608 -0.0778 -0.9287  0.2694  1.3699 -2.0083]
 [ 1.7508 -0.9851  0.      0.      0.      0.      0.      0.      0.    ]
 [-0.2935 -2.4818 -0.1225  0.6199  0.6934 -0.7415  0.      0.      0.    ]
 [ 0.      0.      0.      0.      0.      0.      0.      0.      0.    ]
 [-0.7223 -0.1482  0.7323 -0.837   0.      0.      0.      0.      0.    ]]


---

For a more realistic example, try wrapping [`preprocessing.image.ImageDataGenerator`](https://www.tensorflow.org/api_docs/python/tf/keras/preprocessing/image/ImageDataGenerator) as a [`tf.data.Dataset`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset).

In [27]:
flowers = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)

In [28]:
img_gen = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, rotation_range=20)

In [29]:
flowers

'/home/appuser/.keras/datasets/flower_photos'

In [30]:
%ls $flowers

LICENSE.txt  daisy/  dandelion/  roses/  sunflowers/  tulips/


In [67]:
img_gen.flow_from_directory(flowers)

Found 3670 images belonging to 5 classes.


Iterating `flow_from_directory()` returns a batch of images and labels which are used as input tensors.

In [31]:
images, labels = next(img_gen.flow_from_directory(flowers))

Found 3670 images belonging to 5 classes.


In [32]:
print(images.dtype, images.shape)
print(labels.dtype, labels.shape)

float32 (32, 256, 256, 3)
float32 (32, 5)


In [33]:
ds = tf.data.Dataset.from_generator(
    lambda: img_gen.flow_from_directory(flowers),
    output_types=(tf.float32, tf.float32),
    output_shapes=([32,256,256,3], [32,5])
)

ds.element_spec

(TensorSpec(shape=(32, 256, 256, 3), dtype=tf.float32, name=None),
 TensorSpec(shape=(32, 5), dtype=tf.float32, name=None))

In [34]:
for images, labels in ds.take(1):
  print('images.shape: ', images.shape)
  print('labels.shape: ', labels.shape)

Found 3670 images belonging to 5 classes.
images.shape:  (32, 256, 256, 3)
labels.shape:  (32, 5)


2024-04-24 07:31:31.858226: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Consuming TFRecord data (`tf.data.TFRecordDataset`)

Refer to the [Loading TFRecords](https://www.tensorflow.org/tutorials/load_data/tfrecord) tutorial for an end-to-end example.

The [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) API supports a variety of file formats so that you can process large datasets that do not fit in memory. For example, the TFRecord file format is a simple record-oriented binary format that many TensorFlow applications use for training data. The [`tf.data.TFRecordDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TFRecordDataset) class enables you to stream over the contents of one or more TFRecord files as part of an input pipeline.

Here is an example using the test file from the French Street Name Signs (FSNS).

In [35]:
# Creates a dataset that reads all of the examples from two files.
fsns_test_file = tf.keras.utils.get_file("fsns.tfrec", "https://storage.googleapis.com/download.tensorflow.org/data/fsns-20160927/testdata/fsns-00000-of-00001")

The `filenames` argument to the `TFRecordDataset` initializer can either be a string, a list of strings, or a [`tf.Tensor`](https://www.tensorflow.org/api_docs/python/tf/Tensor) of strings. Therefore if you have two sets of files for training and validation purposes, you can create a factory method that produces the dataset, taking filenames as an input argument:

In [36]:
dataset = tf.data.TFRecordDataset(filenames = [fsns_test_file])
dataset

<TFRecordDatasetV2 element_spec=TensorSpec(shape=(), dtype=tf.string, name=None)>

In [37]:
raw_example = next(iter(dataset))
parsed = tf.train.Example.FromString(raw_example.numpy())

parsed.features.feature['image/text']

bytes_list {
  value: "Rue Perreyon"
}

### Consuming text data (`tf.data.TextLineDataset`)

Refer to the [Load text](https://www.tensorflow.org/tutorials/load_data/text) tutorial for an end-to-end example.

Many datasets are distributed as one or more text files. The [`tf.data.TextLineDataset`](https://www.tensorflow.org/api_docs/python/tf/data/TextLineDataset) provides an easy way to extract lines from one or more text files. Given one or more filenames, a `TextLineDataset` will produce one string-valued element per line of those files.

In [38]:
directory_url = 'https://storage.googleapis.com/download.tensorflow.org/data/illiad/'
file_names = ['cowper.txt', 'derby.txt', 'butler.txt']

file_paths = [
    tf.keras.utils.get_file(file_name, directory_url + file_name)
    for file_name in file_names
]

815980/815980 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
809730/809730 ━━━━━━━━━━━━━━━━━━━━ 0s 1us/step
807992/807992 ━━━━━━━━━━━━━━━━━━━━ 1s 1us/step


In [39]:
dataset = tf.data.TextLineDataset(file_paths)

Here are the first few lines of the first file:

In [40]:
for line in dataset.take(5):
  print(line.numpy())

b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b'His wrath pernicious, who ten thousand woes'
b"Caused to Achaia's host, sent many a soul"
b'Illustrious into Ades premature,'
b'And Heroes gave (so stood the will of Jove)'


2024-04-24 07:31:34.121707: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


To alternate lines between files use [`Dataset.interleave`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#interleave). This makes it easier to shuffle files together. Here are the first, second and third lines from each translation:

In [41]:
files_ds = tf.data.Dataset.from_tensor_slices(file_paths)
lines_ds = files_ds.interleave(tf.data.TextLineDataset, cycle_length=3)

for i, line in enumerate(lines_ds.take(9)):
  if i % 3 == 0:
    print()
  print(line.numpy())


b"\xef\xbb\xbfAchilles sing, O Goddess! Peleus' son;"
b"\xef\xbb\xbfOf Peleus' son, Achilles, sing, O Muse,"
b'\xef\xbb\xbfSing, O goddess, the anger of Achilles son of Peleus, that brought'

b'His wrath pernicious, who ten thousand woes'
b'The vengeance, deep and deadly; whence to Greece'
b'countless ills upon the Achaeans. Many a brave soul did it send'

b"Caused to Achaia's host, sent many a soul"
b'Unnumbered ills arose; which many a soul'
b'hurrying down to Hades, and many a hero did it yield a prey to dogs and'


2024-04-24 07:31:34.144849: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


By default, a `TextLineDataset` yields *every* line of each file, which may not be desirable, for example, if the file starts with a header line, or contains comments. These lines can be removed using the [`Dataset.skip()`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#skip) or [`Dataset.filter`](https://www.tensorflow.org/api_docs/python/tf/data/Dataset#filter) transformations. Here, you skip the first line, then filter to find only survivors.

In [42]:
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_lines = tf.data.TextLineDataset(titanic_file)

In [43]:
for line in titanic_lines.take(10):
  print(line.numpy())

b'survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone'
b'0,male,22.0,1,0,7.25,Third,unknown,Southampton,n'
b'1,female,38.0,1,0,71.2833,First,C,Cherbourg,n'
b'1,female,26.0,0,0,7.925,Third,unknown,Southampton,y'
b'1,female,35.0,1,0,53.1,First,C,Southampton,n'
b'0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y'
b'0,male,2.0,3,1,21.075,Third,unknown,Southampton,n'
b'1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n'
b'1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n'
b'1,female,4.0,1,1,16.7,Third,G,Southampton,n'


2024-04-24 07:31:34.161003: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [44]:
def survived(line):
  return tf.not_equal(tf.strings.substr(line, 0, 1), "0")

survivors = titanic_lines.skip(1).filter(survived)

In [45]:
for line in survivors.take(10):
  print(line.numpy())

b'1,female,38.0,1,0,71.2833,First,C,Cherbourg,n'
b'1,female,26.0,0,0,7.925,Third,unknown,Southampton,y'
b'1,female,35.0,1,0,53.1,First,C,Southampton,n'
b'1,female,27.0,0,2,11.1333,Third,unknown,Southampton,n'
b'1,female,14.0,1,0,30.0708,Second,unknown,Cherbourg,n'
b'1,female,4.0,1,1,16.7,Third,G,Southampton,n'
b'1,male,28.0,0,0,13.0,Second,unknown,Southampton,y'
b'1,female,28.0,0,0,7.225,Third,unknown,Cherbourg,y'
b'1,male,28.0,0,0,35.5,First,A,Southampton,y'
b'1,female,38.0,1,5,31.3875,Third,unknown,Southampton,n'


2024-04-24 07:31:34.193611: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Consuming CSV data

In [46]:
titanic_file = tf.keras.utils.get_file("train.csv", "https://storage.googleapis.com/tf-datasets/titanic/train.csv")

In [47]:
df = pd.read_csv(titanic_file)
df.head()

,survived,sex,age,n_siblings_spouses,parch,fare,class,deck,embark_town,alone
0,0,male,22.0,1,0,7.2500,Third,unknown,Southampton,n
1,1,female,38.0,1,0,71.2833,First,C,Cherbourg,n
2,1,female,26.0,0,0,7.9250,Third,unknown,Southampton,y
3,1,female,35.0,1,0,53.1000,First,C,Southampton,n
4,0,male,28.0,0,0,8.4583,Third,unknown,Queenstown,y


In [48]:
for key, val in dict(df).items():
    print(f'key: {key}\nval: {val}') # val is a Series
    break

key: survived
val: 0      0
1      1
2      1
3      1
4      0
      ..
622    0
623    0
624    1
625    0
626    0
Name: survived, Length: 627, dtype: int64


In [49]:
titanic_slices = tf.data.Dataset.from_tensor_slices(dict(df))

for feature_batch in titanic_slices.take(1):
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

  'survived'          : 0
  'sex'               : b'male'
  'age'               : 22.0
  'n_siblings_spouses': 1
  'parch'             : 0
  'fare'              : 7.25
  'class'             : b'Third'
  'deck'              : b'unknown'
  'embark_town'       : b'Southampton'
  'alone'             : b'n'


2024-04-24 07:31:34.219480: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


---
### (`tf.data.experimental.make_csv_dataset`)
#### more scalable approach

A more scalable approach is to load from disk as necessary.

The [`tf.data`](https://www.tensorflow.org/api_docs/python/tf/data) module provides methods to extract records from one or more CSV files that comply with [RFC 4180](https://tools.ietf.org/html/rfc4180).

The [`tf.data.experimental.make_csv_dataset`](https://www.tensorflow.org/api_docs/python/tf/data/experimental/make_csv_dataset) function is the high-level interface for reading sets of CSV files. It supports column type inference and many other features, like batching and shuffling, to make usage simple.

In [50]:
titanic_batches = tf.data.experimental.make_csv_dataset(
    titanic_file, batch_size=4,
    label_name="survived")

In [51]:
for feature_batch, label_batch in titanic_batches.take(1):
  print("'survived': {}".format(label_batch))
  print("features:")
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

'survived': [0 1 0 1]
features:
  'sex'               : [b'male' b'female' b'male' b'male']
  'age'               : [29. 26. 24. 40.]
  'n_siblings_spouses': [1 0 0 0]
  'parch'             : [0 0 0 0]
  'fare'              : [21.   78.85 79.2  31.  ]
  'class'             : [b'Second' b'First' b'First' b'First']
  'deck'              : [b'unknown' b'unknown' b'B' b'A']
  'embark_town'       : [b'Southampton' b'Southampton' b'Cherbourg' b'Cherbourg']
  'alone'             : [b'n' b'y' b'y' b'y']


2024-04-24 07:31:34.273756: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


You can use the `select_columns` argument if you only need a subset of columns.

In [52]:
titanic_batches = tf.data.experimental.make_csv_dataset(
    titanic_file, batch_size=4,
    label_name="survived", select_columns=['class', 'fare', 'survived'])

In [53]:
for feature_batch, label_batch in titanic_batches.take(1):
  print("'survived': {}".format(label_batch))
  for key, value in feature_batch.items():
    print("  {!r:20s}: {}".format(key, value))

'survived': [0 0 0 1]
  'fare'              : [ 30.       8.6625  33.     512.3292]
  'class'             : [b'First' b'Third' b'Second' b'First']


2024-04-24 07:31:34.310373: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


---

### (`tf.data.experimental.CsvDataset`)
There is also a lower-level [`experimental.CsvDataset`](https://www.tensorflow.org/api_docs/python/tf/data/experimental/CsvDataset) class which provides finer grained control. It does not support column type inference. Instead you must specify the type of each column.

In [54]:
titanic_types  = [tf.int32, tf.string, tf.float32, tf.int32, tf.int32, tf.float32, tf.string, tf.string, tf.string, tf.string]
dataset = tf.data.experimental.CsvDataset(titanic_file, titanic_types , header=True)

for line in dataset.take(10):  # line is a tuple
  print([item.numpy() for item in line])

[0, b'male', 22.0, 1, 0, 7.25, b'Third', b'unknown', b'Southampton', b'n']
[1, b'female', 38.0, 1, 0, 71.2833, b'First', b'C', b'Cherbourg', b'n']
[1, b'female', 26.0, 0, 0, 7.925, b'Third', b'unknown', b'Southampton', b'y']
[1, b'female', 35.0, 1, 0, 53.1, b'First', b'C', b'Southampton', b'n']
[0, b'male', 28.0, 0, 0, 8.4583, b'Third', b'unknown', b'Queenstown', b'y']
[0, b'male', 2.0, 3, 1, 21.075, b'Third', b'unknown', b'Southampton', b'n']
[1, b'female', 27.0, 0, 2, 11.1333, b'Third', b'unknown', b'Southampton', b'n']
[1, b'female', 14.0, 1, 0, 30.0708, b'Second', b'unknown', b'Cherbourg', b'n']
[1, b'female', 4.0, 1, 1, 16.7, b'Third', b'G', b'Southampton', b'n']
[0, b'male', 20.0, 0, 0, 8.05, b'Third', b'unknown', b'Southampton', b'y']


2024-04-24 07:31:34.320811: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


---
If some columns are empty, this low-level interface allows you to provide default values instead of column types.

In [55]:
%%writefile missing.csv
1,2,3,4
,2,3,4
1,,3,4
1,2,,4
1,2,3,
,,,

Overwriting missing.csv


In [56]:
# Creates a dataset that reads all of the records from two CSV files, each with
# four float columns which may have missing values.

record_defaults = [999,999,999,999]
dataset = tf.data.experimental.CsvDataset("missing.csv", record_defaults)

In [57]:
for line in dataset:  # line is a tuple
    print(line)
    for elem in line:  # elem is a tensor
        print(elem.numpy())
        break
    break

(<tf.Tensor: shape=(), dtype=int32, numpy=1>, <tf.Tensor: shape=(), dtype=int32, numpy=2>, <tf.Tensor: shape=(), dtype=int32, numpy=3>, <tf.Tensor: shape=(), dtype=int32, numpy=4>)
1


In [58]:
dataset = dataset.map(lambda *items: tf.stack(items))  # convert a tuple to a tensor
for line in dataset:
    print(line.numpy())

[1 2 3 4]
[999   2   3   4]
[  1 999   3   4]
[  1   2 999   4]
[  1   2   3 999]
[999 999 999 999]


2024-04-24 07:31:34.349482: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


By default, a `CsvDataset` yields *every* column of *every* line of the file, which may not be desirable, for example if the file starts with a header line that should be ignored, or if some columns are not required in the input. These lines and fields can be removed with the `header` and `select_cols` arguments respectively.

In [59]:
# Creates a dataset that reads all of the records from two CSV files with
# headers, extracting float data from columns 2 and 4.
record_defaults = [999, 999] # Only provide defaults for the selected columns
dataset = tf.data.experimental.CsvDataset("missing.csv", record_defaults, select_cols=[1, 3])
dataset = dataset.map(lambda *items: tf.stack(items))
dataset

<_MapDataset element_spec=TensorSpec(shape=(2,), dtype=tf.int32, name=None)>

In [60]:
for line in dataset:
  print(line.numpy())

[2 4]
[2 4]
[999   4]
[2 4]
[  2 999]
[999 999]


2024-04-24 07:31:34.369777: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Consuming sets of files (`tf.data.Dataset.list_files`)

In [61]:
flowers_root = tf.keras.utils.get_file(
    'flower_photos',
    'https://storage.googleapis.com/download.tensorflow.org/example_images/flower_photos.tgz',
    untar=True)
flowers_root = pathlib.Path(flowers_root)

The root directory contains a directory for each class:

In [62]:
for item in flowers_root.glob("*"):
  print(item.name)

sunflowers
daisy
LICENSE.txt
tulips
dandelion
roses


In [63]:
list_ds = tf.data.Dataset.list_files(str(flowers_root/'*/*'))

for f in list_ds.take(5):
  print(f.numpy())

b'/home/appuser/.keras/datasets/flower_photos/dandelion/2479491210_98e41c4e7d_m.jpg'
b'/home/appuser/.keras/datasets/flower_photos/daisy/2488902131_3417698611_n.jpg'
b'/home/appuser/.keras/datasets/flower_photos/dandelion/7196409186_a59957ce0b_m.jpg'
b'/home/appuser/.keras/datasets/flower_photos/tulips/13910544560_9140dd547e.jpg'
b'/home/appuser/.keras/datasets/flower_photos/sunflowers/20658775992_1619cd0a9b_n.jpg'


2024-04-24 07:31:34.394807: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


Read the data using the [`tf.io.read_file`](https://www.tensorflow.org/api_docs/python/tf/io/read_file) function and extract the label from the path, returning `(image, label)` pairs:

In [64]:
def process_path(file_path):
  label = tf.strings.split(file_path, os.sep)[-2]
  return tf.io.read_file(file_path), label

labeled_ds = list_ds.map(process_path)

In [65]:
for image_raw, label_text in labeled_ds.take(1):
  print(repr(image_raw.numpy()[:100]))
  print()
  print(label_text.numpy())

b'\xff\xd8\xff\xe0\x00\x10JFIF\x00\x01\x01\x01\x00H\x00H\x00\x00\xff\xe2\x05XICC_PROFILE\x00\x01\x01\x00\x00\x05Happl\x02 \x00\x00scnrRGB XYZ \x07\xd3\x00\x07\x00\x01\x00\x00\x00\x00\x00\x00acspAPPL\x00\x00\x00\x00appl\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00'

b'tulips'


2024-04-24 07:31:34.450375: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence
